In [1]:
# Source: https://github.com/prestinomills/aqueduct/blob/Know_Your_Community_Pipelines/civis/geohub/ActiveBusinessBlockgroupAggregation/Active_Business_Finalized_Script.py
# %load active_business_script.py
"""
Created on Wed May  1 08:51:03 2019
@author: myrfid041
"""
import geopandas as gpd
import os
import pandas as pd

import esri_credentials

from sodapy import Socrata
from arcgis.gis import GIS
from arcgis.features.summarize_data import join_features
from IPython.display import display
from arcgis.features import FeatureLayer, FeatureLayerCollection, FeatureCollection, SpatialDataFrame
from copy import deepcopy

#---Setting the Outputs
OUTPUT_FILE = "./Listing_of_Active_Businesses.csv"
output_layer_name = "a7236cc62ded454c94a64e9d80d6304a"
max_record_count = 250_000

df = pd.read_pickle("./cleaned_df.p")

layer_rename_columns_dict = {
    'Accommodation and Food Services': 'Accommodation_and_Food_Services',
    'Administrative and Support and Waste Management and Remediation Services': 'Administrative_and_Support_and_',
    'Agriculture, Forestry, Fishing and Hunting': 'Agriculture__Forestry__Fishing_',
    'Arts, Entertainment, and Recreation': 'Arts__Entertainment__and_Recrea',
    'Construction': 'Construction',
    'Educational Services': 'Educational_Services',
    'Finance and Insurance': 'Finance_and_Insurance',
    'Health Care and Social Assistance': 'Health_Care_and_Social_Assistan',
    'Information': 'Information',
    'Manufacturing': 'Manufacturing',
    'Medical Marijuana Collective': 'Medical_Marijuana_Collective',
    'Mining': 'Mining',
    'Not Classified': 'Not_Classified',
    'Other Services (except Public Administration)': 'Other_Services__except_Public_A',
    'Professional, Scientific, and Technical Services': 'Professional__Scientific__and_T',
    'Real Estate Rental and Leasing': 'Real_Estate_Rental_and_Leasing',
    'Retail Trade': 'Retail_Trade',
    'Transportation and Warehousing': 'Transportation_and_Warehousing',
    'Utilities': 'Utilities',
    'Wholesale Trade': 'Wholesale_Trade'                                               
}

lahub_user = esri_credentials.preston_user
lahub_pass = esri_credentials.preston_pw

In [2]:
gis = GIS('https://lahub.maps.arcgis.com',  username=lahub_user, password=lahub_pass)

flayer = gis.content.get(output_layer_name)

ActiveBusinesses_flayer = flayer.layers[0].query()
existing_table = ActiveBusinesses_flayer.sdf

In [3]:
new_updated_table = (df[df.GEOID10.isin(existing_table.GEOID10)]
                     [["GEOID10"] + list(layer_rename_columns_dict.values())]
                    )
UPDATED_VALUES_DICT = new_updated_table.set_index("GEOID10").to_dict(orient="index")

In [4]:
feature_list = ActiveBusinesses_flayer.features
features_to_be_updated = deepcopy(feature_list)

In [ ]:
''' Some are missing...let's just leave those values intact

import numpy as np

existing_list = list(existing_table.GEOID10)
new_list = list(new_updated_table.GEOID10)
missing_geoid = list(np.setdiff1d(existing_list, new_list))
'''

In [13]:
for i in range(0, len(features_to_be_updated)):
    obs = features_to_be_updated[i]
    try:
        geoid = obs.attributes["GEOID10"]
        for col in layer_rename_columns_dict.values():
            obs.attributes[col] = UPDATED_VALUES_DICT[geoid][col]
    except:
        pass

In [17]:
def chunks(l, n, z):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        chunk_list=l[i:i + n]
        z.edit_features(updates= chunk_list)
        print("update successful")

chunks(features_to_be_updated, 1000, flayer.layers[0])

update successful
update successful
update successful
update successful
update successful
update successful
update successful
